# Libraries

In [1]:
import sys

sys.path.insert(0, "/home/leffff/PycharmProjects/LCT_Hack_Yakutiya_2023/venv/lib/python3.10/site-packages")

In [2]:
import os
import random
from joblib import dump, load
from tqdm.auto import tqdm 
tqdm.pandas()
import re 

import pandas as pd
import numpy as np
from datasets import Dataset

import scipy
import sklearn
from sklearn.neighbors import KDTree, NearestNeighbors

import torch
from torch.utils.data import DataLoader
from transformers import AutoModel, AutoTokenizer

In [3]:
def seed_everything(seed: int,
                    use_deterministic_algos: bool = False) -> None:
    
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.use_deterministic_algorithms(use_deterministic_algos)
    random.seed(seed)
    

random_state = 42
seed_everything(random_state)

# Retriever Model

In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "csebuetnlp/mT5_multilingual_XLSum"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/home/leffff/PycharmProjects/LCT_Hack_Yakutiya_2023/venv/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings

In [5]:
class Summarizer:
    def __init__(self, 
                 model: AutoModel,
                 tokenizer: AutoTokenizer,
                 device: str = "cuda"):
        super().__init__()
        self.device = device

        self.model = model
        self.tokenizer = tokenizer
        self.model.eval()
        self.model.to(self.device)

        self.WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))

        
    def summarize(self, text):
        input_ids = self.tokenizer(
            [self.WHITESPACE_HANDLER(text)],
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=512
        )["input_ids"].to(self.device)
        
        output_ids = self.model.generate(
            input_ids=input_ids,
            max_length=256,
            no_repeat_ngram_size=2,
            num_beams=4
        )[0]
        
        summary = self.tokenizer.decode(
            output_ids,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=False
        )

        return summary
                

In [6]:
summarizer = Summarizer(
    model,
    tokenizer
) 
# summarizer.summarize(
#     "от «30 » декабря 2022 г. № Об утверждении Порядка возврата неиспользованных остатков средств, предоставленных муниципальным бюджетным и автономным учреждениям из бюджета МО «Мириинский район» Республики Саха (Якутия)в соответствии с абзацем вторым пункта 1 статьи 78.1. и со статьей 78.2 Бюджетного кодекса Российский Федерации В соответствии с частью 18 статьи 30 Федерального законаот 08.05.2010 № 83-Ф3 «О внесении изменений в отдельные законодательные акты Российской Федерации в связи с совершенствованием правового положения государственных (муниципальных) учреждений», частями 3.17, 3.18 статьи 2 Федерального закона от 03.11.2006 №174-ФЗ «Об автономных учреждениях», постановлениями районной Администрации от 24.11.2020 № 1679 «Об утверждении Порядка определения объема и условий предоставления субсидий на иные цели из бюджета МО «Мирнинский район» Республики Саха (Якутия) муниципальным бюджетным и автономным учреждениям», от 15.02.2017 № 0213 «Об утверждении Положения об осуществлении капитальных вложений в объекты муниципальной собственности МО «Мирнинский район» Республики Саха (Якутия)»: 1. Утвердить Порядок возврата неиспользованных остатков средств, предоставленных муниципальным бюджетным и автономным учреждениям из бюджета МО «Мирнинский район» Республики Саха (Якутия) в соответствии с абзацем вторым пункта | статьи 78.1. и со статьей 78.2 Бюджетного кодекса Российской Федерации согласно приложению к настоящему постановлению. 2. Рекомендовать муниципальным образованиям — поселений 'Мирнинского района руководствоваться настоящим Порядком при разработке муниципальных правовых актов, регламентирующих порядок возврата неиспользованных остатков средств, предоставленных муниципальным бюджетным и автономным учреждениямиз местных бюджетовв соответствии с абзацем вторым пункта | статьи 78.1. и со статьей 78.2 Бюджетного кодекса Российской Федерации. 3. Настоящее Постановление распространяется на правоотношения, возникшиес 01.01.2022 года. 4. Признать утратившим силу постановление районной Администрации от 24.12.2020 № 2059 «Об утверждении Порядка возврата неиспользованных остатков субсидий на иные цели, предоставленных из бюджета МО «Мирнинский район» Республики Саха (Якутия) муниципальным бюджетным и автономным учреждениям». 5. Финансовому управлению (Чемчосва Я.П.) совместно с пресс- службой (Гибало А.О.) разместить настоящее постановление на официальном сайте МО «Мирнинский район» Республики Саха (Якутия) (уу. алмазный край.рф). 6. Контроль исполнения настоящего постановления возложить на заместителя Главы Администрации района по экономике и финансам Башарина Г.К. И.о. Главы Администрации района Д.А. Ширинский Приложение к постановлению районной Администрации от«.4(_» декабря 2022 г.№,03 Порядок возврата неиспользованных остатков средств, предоставленных муниципальным бюджетным и автономным учреждениям из бюджета МО «Мирнинский район»Республики Саха (Якутия) в соответствиис абзацем вторым пункта 1 статьи 78.1 и со статьей 78.2 Бюджетного кодекса Российской Федерации 1. Настоящий Порядок устанавливает правила возврата в бюджет МО «Мирнинский район» Республики Саха (Якутия) неиспользованных на начало текущего финансового года остатков субсидий на иные цели, предоставленных муниципальным бюджетным и автономным учреждениям из бюджета МО «Мирнинский район» Республики Саха (Якутия) в соответствии с абзацем вторым пункта | статьи 78.1. и субсидий на осуществление капитальных вложений в объекты капитального строительства муниципальной собственности и приобретение объектов недвижимого имущества в муниципальную собственность в соответствии со статьей 78.2 Бюджетного кодекса Российской Федерации (далее — субсидии). 2. Возврату подлежат неиспользованные на 1 января текущего финансового года остатки субсидий, предоставленных органами, осуществляющими функции и полномочия учредителя, в течение первых 15 рабочих дней текущего финансового года."
# )

In [7]:
# data = pd.read_csv("parsed.csv")
# data

In [8]:
# data["short_text"] = data["text"].progress_apply(summarizer.summarize)
# data

In [10]:
# data.to_csv("parsed.csv", index=False)

# Dataset Builder

In [11]:
model = AutoModel.from_pretrained("ai-forever/sbert_large_nlu_ru")
tokenizer = AutoTokenizer.from_pretrained("ai-forever/sbert_large_nlu_ru")
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(120138, 1024, padding_idx=0)
    (position_embeddings): Embedding(512, 1024)
    (token_type_embeddings): Embedding(2, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-23): 24 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inp

In [12]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

In [13]:
class RoBERTaEmbeddingBuilder:
    def __init__(self,
                 model: AutoModel,
                 tokenizer: AutoTokenizer,
                 summarizer, 
                 csv_path: str,
                 embeddings_save_path: str = "embeddings.npy",
                 kdtree_save_path: str = "kdtree.joblib",
                 batch_size: int = 1,
                 device: str = "cuda"):
        super().__init__()
        self.device = device

        self.model = model
        self.tokenizer = tokenizer
        self.model.eval()
        self.model.to(self.device)

        self.summarizer = summarizer

        self.embeddings_save_path = embeddings_save_path
        self.kdtree_save_path = kdtree_save_path

        self.batch_size = batch_size

        self._load_dataset(csv_path)
        self._process_dataset()
        self.extract_embeddings()        
    
    def _load_dataset(self, csv_path):
        df = pd.read_csv(csv_path)
        df["short_text"] = df["text"].progress_apply(self.summarizer.summarize)
        self.dataset = Dataset.from_pandas(df)

    def _process_dataset(self):
        self.dataset = self.dataset.map(
            lambda sample: self._preprocess_text(sample['short_text'])
        )

        self.dataset = self.dataset.remove_columns([
            'short_text'
        ])

        self.dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])   
 
    def _preprocess_text(self, text):
        # print(text)
        out = self.tokenizer.encode_plus(text, max_length=512, truncation=True, padding="max_length")
        return out

    def extract_embeddings(self) -> None:
        dataloader = DataLoader(self.dataset, batch_size=self.batch_size)

        embeddings = []

        for batch in tqdm(dataloader):
            input_ids, attention_masks = batch["input_ids"], batch["attention_mask"]
            input_ids, attention_masks = input_ids.to(self.device), attention_masks.to(self.device)

            with torch.no_grad():
                output = mean_pooling(model(
                    input_ids=input_ids,
                    attention_mask=attention_masks,
                ), attention_masks)

            embeddings.append(output.cpu())

        embeddings = torch.cat(embeddings, dim=0).numpy()

        kdtree = NearestNeighbors(n_neighbors=5,
                         metric='cosine',
                         algorithm='brute',
                         n_jobs=-1)
        kdtree.fit(embeddings)
        
        dump(kdtree, self.kdtree_save_path)


In [14]:
builder = RoBERTaEmbeddingBuilder(
    model,
    tokenizer,
    summarizer=summarizer,
    csv_path="parsed.csv",
    embeddings_save_path="embeddings.npy",
    kdtree_save_path="kdtree.joblib",
    batch_size=1,
    device="cuda"
)

  0%|          | 0/457 [00:00<?, ?it/s]

Map:   0%|          | 0/457 [00:00<?, ? examples/s]

  0%|          | 0/457 [00:00<?, ?it/s]

# Indexer

In [15]:
class RoBERTaIndexer:
    def __init__(self, 
                 model: AutoModel, 
                 tokenizer: AutoTokenizer, 
                 k: int, 
                 csv_path: str, 
                 kdtree_load_path: str,
                 device: str = "cuda"):
        super().__init__()
        self.device = device

        self.model = model
        self.model.eval()
        self.model.to(self.device)
        self.tokenizer = tokenizer

        self.df = pd.read_csv(csv_path)

        self.k = k

        self.kdtree = self._load_indexer(kdtree_load_path)
        
    def _load_indexer(self, path):
        kdtree = load(path)
        return kdtree
    
    def _preprocess_text(self, text):
        out = self.tokenizer.encode_plus(text, max_length=512, truncation=True, padding="max_length", return_tensors="pt")
        return out

    def _get_embedding(self, sample):
        input_ids = sample["input_ids"]
        if len(input_ids.shape) < 2:
            input_ids.unsqueeze(dim=0)

        attention_mask = sample["attention_mask"]
        if len(attention_mask.shape) < 2:
            input_ids.unsqueeze(dim=0)

        with torch.no_grad():
            return mean_pooling(self.model(input_ids.to(self.device), attention_mask.to(self.device)), attention_mask.to(self.device)).cpu().numpy()

    def search(self, text):
        sample = self._preprocess_text(text)
        embedding = self._get_embedding(sample)

        ind = self.kdtree.kneighbors(embedding, n_neighbors=5, return_distance=False)
        return self.df.iloc[ind[0]]

In [16]:
indexer = RoBERTaIndexer(
    model, 
    tokenizer,
    k=5,
    csv_path="parsed.csv",
    kdtree_load_path="kdtree.joblib"
)

In [17]:
indexer.search("Планируется ли помощь бездомным?")["text"].values

array(['№442 от«р9» 06 2023г. О проведении общественных обсуждений проектов муниципальных программ В соответствии с Федеральными законами от 06.10.2003 № 131-ФЗ «Об общих принципах организации местного самоуправления в Российской Федерации», от 28.06.2014 №172-ФЗ «О стратегическом планировании в Российской Федерации», статьей 20 Устава муниципального образования «Мирнинский район» Республики Саха (Якутия), в целях общественного обсуждения проектов муниципальных программс участием населения: 1. Провести общественные обсуждения проектов муниципальных программ: МО «Мирнинский район» Республики Саха (Якутия) «Охрана окружающей среды, обращение с отходами производства и потребления» на 2024-2028 годы, «Дополнительное образование в детских школах искусств» на 2024-2028 годы, «Развитие культуры» на 2024-2028 годы, «Создание условий для развития межнациональных и межконфессиональных отношений»на 2024- 2028 годы, «Развитие физической культурыи спорта»на 2024-2028 годы. 2. Назначить: 2.1.дату и 

# Mistral Chat

In [13]:
torch.cuda.empty_cache()

In [14]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig


MODEL_NAME = "IlyaGusev/saiga_mistral_7b"

DEFAULT_MESSAGE_TEMPLATE = "<s>{role}\n{content}</s>"
DEFAULT_RESPONSE_TEMPLATE = "<s>bot\n"
DEFAULT_SYSTEM_PROMPT = "Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им."
# Ты Помогаешь жителям якутии и отвечаеншь на запросы жителей России о планах развития региона."


class Conversation:
    def __init__(
        self,
        message_template=DEFAULT_MESSAGE_TEMPLATE,
        system_prompt=DEFAULT_SYSTEM_PROMPT,
        response_template=DEFAULT_RESPONSE_TEMPLATE
    ):
        self.message_template = message_template
        self.response_template = response_template
        self.messages = [{
            "role": "system",
            "content": system_prompt
        }]

    def add_user_message(self, message):
        self.messages.append({
            "role": "user",
            "content": message
        })

    def add_bot_message(self, message):
        self.messages.append({
            "role": "bot",
            "content": message
        })

    def get_prompt(self, tokenizer):
        final_text = ""
        for message in self.messages:
            message_text = self.message_template.format(**message)
            final_text += message_text
        final_text += DEFAULT_RESPONSE_TEMPLATE
        return final_text.strip()


def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False, max_length=1024)
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(
        **data,
        do_sample=True,
        top_p=0.99,
        top_k=0,
        bos_token_id = 1,
        eos_token_id = 2,
        pad_token_id = 0,
        max_new_tokens=128, 
    )[0]
    print(data["input_ids"].device, model.device)
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()

config = PeftConfig.from_pretrained(MODEL_NAME)
# config.top_k = 1

model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto"
)
model = PeftModel.from_pretrained(
    model,
    MODEL_NAME,
    torch_dtype=torch.float16
)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [15]:
class Chat:
    def __init__(self, model, tokenizer, generation_config, retriever, k: int = 3):
        self.model = model 
        self.tokenizer = tokenizer 
        self.generation_config = generation_config
        self.retriever = retriever
        self.k = k

    def _remove_punct(self, text):
        text = "".join([char for char in text if char.isalpha() or char == " "])
        return text

    def _form_prompt(self, text, retrieved):
        texts = retrieved["text"].tolist()
        prompt = "Ответь на запрос пользователя используя его запрос и документы: " + \
                    "Запрос: " +  text + "\n\n"\
                    "\n".join([f"Документ {i + 1}:\n" + self._remove_punct(texts[i]) for i in range(1)])
        return prompt

    def answer(self, message):
        retrieved_samples = self.retriever.search(message)
        prompt = self._form_prompt(message, retrieved_samples)
        
        conversation = Conversation()
        conversation.add_user_message(prompt)
        print(conversation.messages)
        prompt = conversation.get_prompt(tokenizer)

        output = generate(self.model, self.tokenizer, prompt, self.generation_config)

        return output

In [16]:
torch.cuda.empty_cache()

In [17]:
chat = Chat(model, tokenizer, generation_config, indexer)

In [18]:
chat.answer("Планируется ли строительство гаражей?")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[{'role': 'system', 'content': 'Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им.'}, {'role': 'user', 'content': 'Ответь на запрос пользователя используя его запрос и документы: Запрос: Планируется ли строительство гаражей?Документ 1:\nг Мирный от И    ИЗ Об утверждении муниципальной программы МО Мирнинский районРС Я Создание условий для оказания медицинской помощи населению и укрепления общественного здоровья В соответствии состатьей  Бюджетного кодекса Российской Федерации Федеральным законом от   ФЗ О стратегическом планировании в Российской Федерации Уставом муниципального образования Мирнинский район Республики Саха Якутия постановлением районной Администрации от    Об утверждении Порядка разработки реализации и оценки эффективности муниципальных программ МО Мирнинский район Республики Саха Якутия  Утвердить муниципальную программу МО Мирнинский районРС Я Создание условий для оказания медицинской помощи населению и укрепления общественн

'ний  Повышение качества и безопасности оказания медицинской помощи населению\nВозвращаюсь. Возможно, не能 разобраться точно в запросе пользователя, но по последовательности, кажется, следует ответить, что документы не содержат никаких упоминаний о строительстве гаражей.'

# FRED T5 QA Chat

In [18]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
from transformers import GenerationConfig

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
generation_config = GenerationConfig.from_pretrained("Den4ikAI/FRED-T5-LARGE_text_qa")
generation_config.num_beams = 10
generation_config.seed=42
tokenizer = AutoTokenizer.from_pretrained("Den4ikAI/FRED-T5-LARGE_text_qa")
model = AutoModelForSeq2SeqLM.from_pretrained("Den4ikAI/FRED-T5-LARGE_text_qa").to(device)
model.eval()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


T5ForConditionalGeneration(
  (shared): Embedding(50364, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(50364, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=1024, out_features=2816, bias=False)
              (wi_1): Linear(in_features=1024, out_features=2816, bias=False)
       

In [19]:
class Chat:
    def __init__(self, model, tokenizer, generation_config, retriever, k: int = 3):
        self.model = model 
        self.tokenizer = tokenizer 
        self.generation_config = generation_config
        self.retriever = retriever
        self.k = k

    def _remove_punct(self, text):
        text = "".join([char for char in text if char.isalpha() or char == " "])
        return text

    def _form_prompt(self, text, retrieved):
        texts = retrieved["text"].tolist()

        prompt = f"<SC6>Текст: {texts[0] + ' Ответь развернуто'}\nВопрос: {text}\nОтвет: <extra_id_0>"
        
        # prompt = "Ответь на запрос пользователя используя его запрос и документы: " + \
        #             "Запрос: " +  text + "\n\n"\
        #             "\n".join([f"Документ {i + 1}:\n" + self._remove_punct(texts[i]) for i in range(1)])
        return prompt
        
    def generate(self, prompt):
      data = self.tokenizer(prompt, return_tensors="pt").to(model.device)
      output_ids = self.model.generate(
          **data,
          generation_config=self.generation_config,
      )[0]
      # print(self.tokenizer.decode(data["input_ids"][0].tolist()))
      out = self.tokenizer.decode(output_ids.tolist(), skip_special_tokens=True)
      return out

    def answer(self, message):
        retrieved_samples = self.retriever.search(message)
        prompt = self._form_prompt(message, retrieved_samples)
        
        return self.generate(prompt)[13:], f"На основе документа {retrieved_samples['url'].values[0]}"

In [20]:
chat = Chat(model, tokenizer, generation_config, indexer)

In [21]:
chat.answer("Планируется ли открытие новых спорт комплексов?")

('Да, планируется открытие новых спорт комплексов в Мирнинском районе.',
 'На основе документа https://www.алмазный-край.рф//upload/files/bnr/885.pdf')

In [22]:
chat.answer("Что по поводу поддержки военнослужащих?")

('Постановление от 07.09.2022 №1231 "Об утверждении Порядка оказания единовременной материальной помощи членам семей военнослужащих, а также лиц, проходящих службу в войсках национальной гвардии Российской Федерации и имеющих звание полиции, погибших (умерших) в ходе проведения специальной военной операции на территории Донецкой Народной Республики, Луганской Народной Республики и Украины"',
 'На основе документа https://www.алмазный-край.рф//upload/files/bnr/1%20ЛДНР.pdf')

In [23]:
chat.answer("Что по поводу помощи детям сиротам?")

('"Об утверждении Положения об оказании материальной помощи первоклассникам, выпускникам, муниципальных общеобразовательных организаций Мирнинского района, относящимся к категории детей-сирот и детей, оставшихся без попечения родителей, лиц из числа детей',
 'На основе документа https://www.алмазный-край.рф//1156.pdf')

In [24]:
chat.answer("Планируется ли помощь детям сиротам?")

('Нет_\nВ соответствии с Федеральным законом от 24.04.2008 № 48-ФЗ «Об опеке и попечительстве», муниципальной программой МО «Мирнинский район» РС(Я) «Социальные меры реабилитации детей-сирот и детей, оставшихся без попечения родителей, в Мирнинском районе» на 2019-2023 годы, планируется оказание материальной помощи детям-сиротам.',
 'На основе документа https://www.алмазный-край.рф//1156.pdf')